# ENDF Data Retrieval and Caching in PLEIADES

This notebook demonstrates how to use the PLEIADES nuclear data management system to efficiently retrieve and cache ENDF (Evaluated Nuclear Data File) resonance data for nuclear calculations.

## What You'll Learn

1. How to configure the PLEIADES nuclear data cache
2. Two different methods for retrieving ENDF data:
   - **DIRECT Method**: Downloads complete ENDF files and extracts resonance data
   - **API Method**: Uses the IAEA EXFOR API to download only the resonance section

## Benefits of Each Approach

- **DIRECT Method**: 
  - Gets the complete ENDF file (all sections)
  - Useful when you need the entire dataset
  - Higher storage requirements

- **API Method**:
  - Downloads only the resonance data section
  - Faster and more efficient
  - Lower storage requirements
  - Ideal when you only need resonance parameters

In [1]:
import time
import shutil
from pathlib import Path
from pleiades.nuclear.manager import NuclearDataManager, EndfLibrary
from pleiades.nuclear.models import DataRetrievalMethod
from pleiades.utils.config import get_config
from pleiades.utils.logger import configure_logger, loguru_logger

# Configure logging to see debug information
configure_logger(console_level="INFO")

# Set up logger for this notebook
logger = loguru_logger.bind(name="endf_retrieval_notebook")

2025-04-28 16:56:56 | INFO     | pleiades.utils.logger:configure_logger:95 - Logging configured. Log file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/pleiades_20250428_165656.log


In [2]:
# Get configuration and show available settings
config = get_config()
logger.info(f"Cache directory: {config.nuclear_data_cache_dir}")
logger.info(f"Available data sources: {config.nuclear_data_sources}")

# Create output directory for parameter files
output_dir = Path("./endf_test")
output_dir.mkdir(exist_ok=True)
logger.info(f"Output directory: {output_dir.absolute()}")

# Option to clear the cache (set to True to clear)
CLEAR_CACHE = True  # Change to True if you want to clear the cache

if CLEAR_CACHE:
    user_input = input("This will delete all cached ENDF files. Are you sure? (yes/no): ")
    if user_input.lower() == "yes":
        # Remove all files in the cache directory
        if config.nuclear_data_cache_dir.exists():
            shutil.rmtree(config.nuclear_data_cache_dir)
            logger.warning("Cache directory cleared.")
            # Recreate the empty directory
            config.nuclear_data_cache_dir.mkdir(parents=True, exist_ok=True)
    else:
        logger.info("Cache clearing skipped. Note that notebook outputs may differ from descriptions due to existing cached files.")

2025-04-28 16:56:58 | INFO     | __main__:<module>:3 - Cache directory: /Users/8cz/.pleiades/nuclear_data
2025-04-28 16:56:58 | INFO     | __main__:<module>:4 - Available data sources: {'DIRECT': 'https://www-nds.iaea.org/public/download-endf', 'API': 'https://www-nds.iaea.org/exfor/servlet'}
2025-04-28 16:56:58 | INFO     | __main__:<module>:9 - Output directory: /Users/8cz/github.com/PLEIADES/examples/Notebooks/endf_test
2025-04-28 16:56:58 | INFO     | __main__:<module>:4 - Available data sources: {'DIRECT': 'https://www-nds.iaea.org/public/download-endf', 'API': 'https://www-nds.iaea.org/exfor/servlet'}
2025-04-28 16:56:58 | INFO     | __main__:<module>:9 - Output directory: /Users/8cz/github.com/PLEIADES/examples/Notebooks/endf_test
2025-04-28 16:57:10 | WARNING  | __main__:<module>:20 - Cache directory cleared.
2025-04-28 16:57:10 | WARNING  | __main__:<module>:20 - Cache directory cleared.


In [3]:
# Initialize NuclearDataManager
nuclear_manager = NuclearDataManager()
logger.info(f"Default ENDF library: {nuclear_manager.default_library}")

# List cache directories
cache_root = config.nuclear_data_cache_dir
logger.info("Cache structure:")
if cache_root.exists():
    for method_dir in cache_root.iterdir():
        if method_dir.is_dir():
            logger.info(f"  Method: {method_dir.name}")
            for library_dir in method_dir.iterdir():
                if library_dir.is_dir():
                    file_count = len(list(library_dir.glob("*.dat")))
                    logger.info(f"    Library: {library_dir.name} ({file_count} files)")

2025-04-28 16:57:11 | INFO     | __main__:<module>:3 - Default ENDF library: EndfLibrary.ENDF_B_VIII_0
2025-04-28 16:57:11 | INFO     | __main__:<module>:7 - Cache structure:
2025-04-28 16:57:11 | INFO     | __main__:<module>:11 -   Method: DataRetrievalMethod.DIRECT
2025-04-28 16:57:11 | INFO     | __main__:<module>:15 -     Library: EndfLibrary.JENDL_5 (0 files)
2025-04-28 16:57:11 | INFO     | __main__:<module>:15 -     Library: EndfLibrary.TENDL_2021 (0 files)
2025-04-28 16:57:11 | INFO     | __main__:<module>:15 -     Library: EndfLibrary.JEFF_3_3 (0 files)
2025-04-28 16:57:11 | INFO     | __main__:<module>:15 -     Library: EndfLibrary.ENDF_B_VIII_1 (0 files)
2025-04-28 16:57:11 | INFO     | __main__:<module>:15 -     Library: EndfLibrary.ENDF_B_VIII_0 (0 files)
2025-04-28 16:57:11 | INFO     | __main__:<module>:15 -     Library: EndfLibrary.CENDL_3_2 (0 files)
2025-04-28 16:57:11 | INFO     | __main__:<module>:11 -   Method: DataRetrievalMethod.API
2025-04-28 16:57:11 | INFO    

## Isotope Setup

First, we'll load information about the U-238 isotope which will be used throughout this tutorial.

In [4]:
isotope_str = "U-238"  # Test with U-238
logger.info(f"Testing with isotope: {isotope_str}")

# Get isotope info
isotope_info = nuclear_manager.isotope_manager.get_isotope_info(isotope_str)
logger.info(f"Isotope details: {isotope_info}")

2025-04-28 16:57:20 | INFO     | __main__:<module>:2 - Testing with isotope: U-238
2025-04-28 16:57:20 | INFO     | __main__:<module>:6 - Isotope details: IsotopeInfo class for: U-238
2025-04-28 16:57:20 | INFO     | __main__:<module>:6 - Isotope details: IsotopeInfo class for: U-238


Searching for mass.mas20 in cached files for FileCategory.ISOTOPES: {PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/neutrons.list'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20')}
Checking file: isotopes.info
Checking file: neutrons.list
Checking file: mass.mas20
Found file: /Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20
Searching for isotopes.info in cached files for FileCategory.ISOTOPES: {PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/neutrons.list'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20')}
Checking file: isotopes.info
Found file: /Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info

## Method 1: DIRECT Download

This method downloads the complete ENDF file and extracts only the resonance parameters.
- Downloads entire ENDF file (all sections)
- Extracts only the resonance parameter lines
- Stores the complete file in cache for future use

In [5]:
%%time
# Download and extract resonance file (first time should download)
logger.info("\nDIRECT Method - First download (should fetch from remote):")
output_path = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    method=DataRetrievalMethod.DIRECT,
    use_cache=True
)
logger.info(f"Output file: {output_path}")

2025-04-28 16:57:22 | INFO     | __main__:<module>:2 - 
DIRECT Method - First download (should fetch from remote):
2025-04-28 16:57:22 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:196 - Downloading complete ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.0/n/n_9237_92-U-238.zip
2025-04-28 16:57:22 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:197 - This will download and cache the entire ENDF file (all sections)
2025-04-28 16:57:22 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:196 - Downloading complete ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.0/n/n_9237_92-U-238.zip
2025-04-28 16:57:22 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:197 - This will download and cache the entire ENDF file (all sections)
2025-04-28 16:57:24 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:376 - Extracting resonance parameters from complete ENDF file
2025-04-28 16:57:24 | INFO

CPU times: user 82.5 ms, sys: 48.7 ms, total: 131 ms
Wall time: 1.43 s


In [6]:
# Examine the cached file
cache_file_path = nuclear_manager._get_cache_file_path(
      method=DataRetrievalMethod.DIRECT,
      library=EndfLibrary.ENDF_B_VIII_0,
      isotope=isotope_info,
      mat=isotope_info.material_number
  )
logger.info(f"Cache file path: {cache_file_path}")
logger.info(f"Exists in cache: {cache_file_path.exists()}")

if cache_file_path.exists():
    # Get file size
    file_size = cache_file_path.stat().st_size
    logger.info(f"Cache file size: {file_size / 1024:.2f} KB")

    # Preview first few lines
    logger.info("\nFirst 5 lines of cached file:")
    with open(cache_file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            logger.info(line.strip())

2025-04-28 16:57:25 | INFO     | __main__:<module>:8 - Cache file path: /Users/8cz/.pleiades/nuclear_data/DataRetrievalMethod.DIRECT/EndfLibrary.ENDF_B_VIII_0/n_9237_92-U-238.dat
2025-04-28 16:57:25 | INFO     | __main__:<module>:9 - Exists in cache: True
2025-04-28 16:57:25 | INFO     | __main__:<module>:14 - Cache file size: 15741.04 KB
2025-04-28 16:57:25 | INFO     | __main__:<module>:17 - 
First 5 lines of cached file:
2025-04-28 16:57:25 | INFO     | __main__:<module>:22 - Retrieved by E4-util: 2018/02/07,18:09:12                            1 0  0    0
2025-04-28 16:57:25 | INFO     | __main__:<module>:22 - 9.223800+4 2.360058+2          1          1          0          19237 1451    1
2025-04-28 16:57:25 | INFO     | __main__:<module>:22 - 0.000000+0 0.000000+0          0          0          0          69237 1451    2
2025-04-28 16:57:25 | INFO     | __main__:<module>:22 - 1.000000+0 3.000000+7          0          0         10          89237 1451    3
2025-04-28 16:57:25 | INFO 

In [7]:
%%time

logger.info("\nDIRECT Method - Second download (should use cache):")
output_path2 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    method=DataRetrievalMethod.DIRECT,
    use_cache=True
)
logger.info(f"Output file: {output_path2}")

# Check output file size
output_size = output_path2.stat().st_size
logger.info(f"Output file size: {output_size / 1024:.2f} KB (only resonance parameters)")

# Compare with full ENDF file
full_file_size = cache_file_path.stat().st_size if cache_file_path.exists() else 0
if full_file_size > 0:
    logger.info(f"Size difference: The resonance-only file is {output_size/full_file_size*100:.2f}% of the full ENDF file size")

2025-04-28 16:57:33 | INFO     | __main__:<module>:1 - 
DIRECT Method - Second download (should use cache):
2025-04-28 16:57:33 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:361 - Using cached ENDF data from /Users/8cz/.pleiades/nuclear_data/DataRetrievalMethod.DIRECT/EndfLibrary.ENDF_B_VIII_0/n_9237_92-U-238.dat
2025-04-28 16:57:33 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:376 - Extracting resonance parameters from complete ENDF file
2025-04-28 16:57:33 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:361 - Using cached ENDF data from /Users/8cz/.pleiades/nuclear_data/DataRetrievalMethod.DIRECT/EndfLibrary.ENDF_B_VIII_0/n_9237_92-U-238.dat
2025-04-28 16:57:33 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:376 - Extracting resonance parameters from complete ENDF file
2025-04-28 16:57:33 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:385 - Resonance parameters extracted and written to

CPU times: user 34.2 ms, sys: 11.1 ms, total: 45.3 ms
Wall time: 44.1 ms


## Method 2: API-Based Retrieval

This method uses the IAEA EXFOR API to directly download only the resonance section of the ENDF file.
- Only downloads the neutron resonance section (much smaller file)
- More efficient when you only need resonance parameters
- Still supports caching for improved performance on repeated use

In [8]:
# Examine the API cached file
api_cache_file_path = nuclear_manager._get_cache_file_path(
      method=DataRetrievalMethod.API,
      library=EndfLibrary.ENDF_B_VIII_0,
      isotope=isotope_info,
      mat=isotope_info.material_number
  )
logger.info(f"API cache file path: {api_cache_file_path}")
logger.info(f"Exists in cache: {api_cache_file_path.exists()}")

if api_cache_file_path.exists():
    # Get file size
    file_size = api_cache_file_path.stat().st_size
    logger.info(f"API cache file size: {file_size / 1024:.2f} KB")

    # Compare with DIRECT method cache size
    if cache_file_path.exists():
        direct_size = cache_file_path.stat().st_size
        logger.info(f"Size comparison: API cache is {file_size/direct_size*100:.2f}% the size of DIRECT cache")

    # Preview first few lines
    logger.info("\nFirst 5 lines of API cached file:")
    with open(api_cache_file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            logger.info(line.strip())

2025-04-28 16:57:35 | INFO     | __main__:<module>:8 - API cache file path: /Users/8cz/.pleiades/nuclear_data/DataRetrievalMethod.API/EndfLibrary.ENDF_B_VIII_0/n_092-U-238_9237_resonance.dat
2025-04-28 16:57:35 | INFO     | __main__:<module>:9 - Exists in cache: False
2025-04-28 16:57:35 | INFO     | __main__:<module>:9 - Exists in cache: False


In [9]:
%%time
# Second download using API method (should use cache)
logger.info("\nAPI Method - Second download (should use cache):")
output_path_api2 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    method=DataRetrievalMethod.API,
    use_cache=True
)
logger.info(f"Output file: {output_path_api2}")

# Check output file size
api_output_size = output_path_api2.stat().st_size if output_path_api2.exists() else 0
if api_output_size > 0 and output_size > 0:
    logger.info(f"Output file comparison: Both API and DIRECT methods produce the same resonance-only output files")
    logger.info(f"API output size: {api_output_size / 1024:.2f} KB")
    logger.info(f"DIRECT output size: {output_size / 1024:.2f} KB")

2025-04-28 16:57:36 | INFO     | __main__:<module>:2 - 
API Method - Second download (should use cache):
2025-04-28 16:57:36 | INFO     | pleiades.nuclear.manager:_get_data_from_api:251 - Searching for neutron resonance data for U-238 in ENDF/B-VIII.0 via IAEA EXFOR API
2025-04-28 16:57:36 | INFO     | pleiades.nuclear.manager:_get_data_from_api:252 - Note: This will download ONLY the resonance section, not the complete ENDF file
2025-04-28 16:57:36 | INFO     | pleiades.nuclear.manager:_get_data_from_api:251 - Searching for neutron resonance data for U-238 in ENDF/B-VIII.0 via IAEA EXFOR API
2025-04-28 16:57:36 | INFO     | pleiades.nuclear.manager:_get_data_from_api:252 - Note: This will download ONLY the resonance section, not the complete ENDF file
2025-04-28 16:57:37 | INFO     | pleiades.nuclear.manager:_get_data_from_api:282 - Found neutron resonance data (SectID: 19173028) for U-238 in INDEN-Aug2023
2025-04-28 16:57:37 | INFO     | pleiades.nuclear.manager:_get_data_from_api:28

CPU times: user 27.5 ms, sys: 12.5 ms, total: 40 ms
Wall time: 1.47 s


## Performance Comparison

Let's compare the performance of both methods by downloading data for another isotope (U-235).
We'll measure the time taken for each method and examine the cache sizes.

In [10]:
isotope_str2 = "U-235"
logger.info(f"\nTesting with another isotope: {isotope_str2}")

isotope_info2 = nuclear_manager.isotope_manager.get_isotope_info(isotope_str2)
logger.info(f"Isotope details: {isotope_info2}")

2025-04-28 16:57:39 | INFO     | __main__:<module>:2 - 
Testing with another isotope: U-235
2025-04-28 16:57:39 | INFO     | __main__:<module>:5 - Isotope details: IsotopeInfo class for: U-235
2025-04-28 16:57:39 | INFO     | __main__:<module>:5 - Isotope details: IsotopeInfo class for: U-235


In [11]:
%%time
# Download U-235 data using DIRECT method
logger.info("Downloading U-235 data using DIRECT method...")
start_time = time.time()
output_direct_u235 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info2,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    method=DataRetrievalMethod.DIRECT,
    use_cache=True
)
direct_time = time.time() - start_time
logger.info(f"DIRECT method time: {direct_time:.2f} seconds")
logger.info(f"Output file: {output_direct_u235}")

2025-04-28 16:57:40 | INFO     | __main__:<module>:2 - Downloading U-235 data using DIRECT method...
2025-04-28 16:57:40 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:196 - Downloading complete ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.0/n/n_9228_92-U-235.zip
2025-04-28 16:57:40 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:197 - This will download and cache the entire ENDF file (all sections)
2025-04-28 16:57:40 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:196 - Downloading complete ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.0/n/n_9228_92-U-235.zip
2025-04-28 16:57:40 | INFO     | pleiades.nuclear.manager:_get_data_from_direct:197 - This will download and cache the entire ENDF file (all sections)
2025-04-28 16:57:42 | INFO     | pleiades.nuclear.manager:download_endf_resonance_file:376 - Extracting resonance parameters from complete ENDF file
2025-04-28 16:57:42 | INFO     | pleiade

CPU times: user 189 ms, sys: 144 ms, total: 333 ms
Wall time: 1.84 s


In [12]:
%%time
# Download U-235 data using API method
logger.info("Downloading U-235 data using API method...")
start_time = time.time()
output_api_u235 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info2,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    method=DataRetrievalMethod.API,
    use_cache=True
)
api_time = time.time() - start_time
logger.info(f"API method time: {api_time:.2f} seconds")
logger.info(f"Output file: {output_api_u235}")

# Compare times
if direct_time > 0 and api_time > 0:
    logger.info(f"\nPerformance comparison:")
    logger.info(f"DIRECT method: {direct_time:.2f} seconds")
    logger.info(f"API method: {api_time:.2f} seconds")
    
    if direct_time > api_time:
        logger.info(f"API method was {direct_time/api_time:.2f}x faster")
    else:
        logger.info(f"DIRECT method was {api_time/direct_time:.2f}x faster")

2025-04-28 16:57:43 | INFO     | __main__:<module>:2 - Downloading U-235 data using API method...
2025-04-28 16:57:43 | INFO     | pleiades.nuclear.manager:_get_data_from_api:251 - Searching for neutron resonance data for U-235 in ENDF/B-VIII.0 via IAEA EXFOR API
2025-04-28 16:57:43 | INFO     | pleiades.nuclear.manager:_get_data_from_api:252 - Note: This will download ONLY the resonance section, not the complete ENDF file
2025-04-28 16:57:43 | INFO     | pleiades.nuclear.manager:_get_data_from_api:251 - Searching for neutron resonance data for U-235 in ENDF/B-VIII.0 via IAEA EXFOR API
2025-04-28 16:57:43 | INFO     | pleiades.nuclear.manager:_get_data_from_api:252 - Note: This will download ONLY the resonance section, not the complete ENDF file
2025-04-28 16:57:44 | INFO     | pleiades.nuclear.manager:_get_data_from_api:282 - Found neutron resonance data (SectID: 19172584) for U-235 in INDEN-Aug2023
2025-04-28 16:57:44 | INFO     | pleiades.nuclear.manager:_get_data_from_api:282 - Fou

CPU times: user 30.1 ms, sys: 13.1 ms, total: 43.2 ms
Wall time: 1.54 s


## Cache Structure Summary

Let's examine the cache directory structure to see how files are organized and how the two methods compare in terms of storage requirements.

In [13]:
logger.info("Cache content summary with filename patterns:")
cache_root = config.nuclear_data_cache_dir
total_direct_size = 0
total_api_size = 0

if cache_root.exists():
    for method_dir in cache_root.iterdir():
        if method_dir.is_dir():
            method_size = 0
            logger.info(f"\nMethod: {method_dir.name}")
            for library_dir in method_dir.iterdir():
                if library_dir.is_dir():
                    files = list(library_dir.glob("*.dat"))
                    lib_size = sum(f.stat().st_size for f in files)
                    method_size += lib_size
                    
                    if files:
                        logger.info(f"  Library: {library_dir.name}")
                        logger.info(f"  Total size: {lib_size / 1024:.2f} KB")
                        for file in files:
                            file_size = file.stat().st_size
                            logger.info(f"    - {file.name} ({file_size / 1024:.2f} KB)")
                            
                            # Check filename pattern
                            if "_" in file.name and file.name.split("_")[1].isdigit():
                                logger.debug(f"      Pattern: MAT_FIRST (non-zero-padded Z)")
                            else:
                                logger.debug(f"      Pattern: ELEMENT_FIRST (zero-padded Z)")
            
            # Add to method totals
            if method_dir.name == 'DIRECT':
                total_direct_size = method_size
            elif method_dir.name == 'API':
                total_api_size = method_size
            
            logger.info(f"  Total {method_dir.name} cache size: {method_size / (1024*1024):.2f} MB")
    
    # Overall comparison
    if total_direct_size > 0 and total_api_size > 0:
        logger.info(f"\nOverall storage comparison:")
        logger.info(f"Total DIRECT cache: {total_direct_size / (1024*1024):.2f} MB")
        logger.info(f"Total API cache: {total_api_size / (1024*1024):.2f} MB")
        logger.info(f"API method uses {total_api_size/total_direct_size*100:.2f}% of the DIRECT method's storage")

2025-04-28 16:57:47 | INFO     | __main__:<module>:1 - Cache content summary with filename patterns:
2025-04-28 16:57:47 | INFO     | __main__:<module>:10 - 
Method: DataRetrievalMethod.DIRECT
2025-04-28 16:57:47 | INFO     | __main__:<module>:18 -   Library: EndfLibrary.ENDF_B_VIII_0
2025-04-28 16:57:47 | INFO     | __main__:<module>:19 -   Total size: 54604.15 KB
2025-04-28 16:57:47 | INFO     | __main__:<module>:22 -     - n_9228_92-U-235.dat (38863.12 KB)
2025-04-28 16:57:47 | INFO     | __main__:<module>:22 -     - n_9237_92-U-238.dat (15741.04 KB)
2025-04-28 16:57:47 | INFO     | __main__:<module>:36 -   Total DataRetrievalMethod.DIRECT cache size: 53.32 MB
2025-04-28 16:57:47 | INFO     | __main__:<module>:10 - 
Method: DataRetrievalMethod.API
2025-04-28 16:57:47 | INFO     | __main__:<module>:18 -   Library: EndfLibrary.ENDF_B_VIII_0
2025-04-28 16:57:47 | INFO     | __main__:<module>:19 -   Total size: 539.63 KB
2025-04-28 16:57:47 | INFO     | __main__:<module>:22 -     - n_09

## Conclusion: Choosing the Right Method

Both methods have their advantages:

- **DIRECT Method**:
  - Downloads the complete ENDF file
  - Good when you need access to other sections beyond resonance data
  - Higher storage requirements
  - May take longer for initial download

- **API Method**:
  - Downloads only the resonance section
  - Faster downloads and lower storage requirements
  - Ideal when you only need resonance parameters for calculations
  - Particularly useful for large-scale processing of many isotopes

### Usage Guidelines:
- Use **DIRECT** when you need complete ENDF files or multiple sections
- Use **API** when you only need resonance parameters and want to optimize performance and storage

Both methods benefit from caching, which significantly speeds up subsequent retrievals.